In [1]:
# 1. ИМПОРТЫ И НАСТРОЙКИ
from pathlib import Path
import json
import re
import xml.etree.ElementTree as ET
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Импорты для работы с PDF и DOC
try:
    import PyPDF2
    import pdfplumber
    print("✅ PDF библиотеки загружены")
except ImportError:
    print("❌ Установите: pip install PyPDF2 pdfplumber")

try:
    from docx import Document
    print("✅ DOC библиотеки загружены")
except ImportError:
    print("❌ Установите: pip install python-docx")

print("📚 СИСТЕМА ОБРАБОТКИ КНИГ ПО ПРОДАЖАМ")
print("=" * 50)
print("Поддерживаемые форматы: FB2, PDF, DOC, DOCX")
print("Результат: CSV файл с техниками продаж")
print("=" * 50)


❌ Установите: pip install PyPDF2 pdfplumber
✅ DOC библиотеки загружены
📚 СИСТЕМА ОБРАБОТКИ КНИГ ПО ПРОДАЖАМ
Поддерживаемые форматы: FB2, PDF, DOC, DOCX
Результат: CSV файл с техниками продаж


In [2]:
# 2. КЛАСС SALESBOOKPROCESSOR - ОСНОВА СИСТЕМЫ
class SalesBookProcessor:
    """Процессор книг по продажам - извлекает техники из FB2, PDF, DOC файлов"""
    
    def __init__(self):
        self.books_dir = Path("books")
        self.data_dir = Path("data")
        self.data_dir.mkdir(exist_ok=True)
        
        # Паттерны для поиска техник продаж
        self.technique_patterns = {
            'выявление_потребностей': [
                r'[А-ЯЁ][^.!?]*(?:что|как|почему|когда|где|какой|какая|какие)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:расскажите|объясните|опишите|поделитесь)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:важно|интересует|волнует|беспокоит)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:потребност|нужд|требован)[^.!?]*[.!?]'
            ],
            'презентация_объектов': [
                r'[А-ЯЁ][^.!?]*(?:это означает что|благодаря этому|в результате)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:представьте себе|посмотрите|обратите внимание)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:выгода|преимущество|польза)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:квартира|дом|объект)[^.!?]*(?:характеристик|особенност)[^.!?]*[.!?]'
            ],
            'работа_с_возражениями': [
                r'[А-ЯЁ][^.!?]*(?:понимаю|согласен)[^.!?]*(?:но|однако|тем не менее)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:возражение|сомнение)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:дорого)[^.!?]*[.!?]'
            ],
            'установление_доверия': [
                r'[А-ЯЁ][^.!?]*(?:опыт показывает|практика показывает)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:другие клиенты|наши клиенты)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:гарантирую|обещаю|ручаюсь)[^.!?]*[.!?]',
                r'[А-ЯЁ][^.!?]*(?:доверие|надежность)[^.!?]*[.!?]'
            ],
            'закрытие_сделки': [
                r'[А-ЯЁ][^.!?]*(?:готовы|согласны)[^.!?]*(?:подписать|оформить|купить)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:когда удобно|когда вам подходит)[^.!?]*\?',
                r'[А-ЯЁ][^.!?]*(?:выбираете|принимаете решение)[^.!?]*\?'
            ]
        }
        
        print("✅ Класс SalesBookProcessor инициализирован")
        print(f"📁 Папка с книгами: {self.books_dir}")
        print(f"💾 Папка для данных: {self.data_dir}")
        print(f"🏷️  Категории техник: {len(self.technique_patterns)}")

print("🔧 Класс определён, готов к использованию")


🔧 Класс определён, готов к использованию


In [3]:
# 3. МЕТОДЫ ИЗВЛЕЧЕНИЯ ТЕКСТА ИЗ РАЗНЫХ ФОРМАТОВ

# Добавляем методы к классу SalesBookProcessor
def extract_text_from_file(self, file_path):
    """Главный метод извлечения текста в зависимости от формата файла"""
    file_path = Path(file_path)
    extension = file_path.suffix.lower()
    
    print(f"🔍 Обрабатываю: {file_path.name} ({extension})")
    
    if extension == '.fb2':
        return self.extract_fb2_text(file_path)
    elif extension == '.pdf':
        return self.extract_pdf_text(file_path)
    elif extension in ['.doc', '.docx']:
        return self.extract_doc_text(file_path)
    else:
        print(f"❌ Неподдерживаемый формат: {extension}")
        return ""

def extract_fb2_text(self, file_path):
    """Извлечение текста из FB2 файла через XML, regex и binary методы"""
    try:
        # Метод 1: XML парсинг
        tree = ET.parse(file_path)
        root = tree.getroot()
        
        text_parts = []
        for elem in root.iter():
            if elem.text and elem.text.strip():
                text_parts.append(elem.text.strip())
        
        if text_parts:
            return ' '.join(text_parts)
    except Exception as e:
        print(f"XML метод не работает: {e}")
    
    try:
        # Метод 2: Regex поиск
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        text_matches = re.findall(r'<p[^>]*>(.*?)</p>', content, re.DOTALL)
        if text_matches:
            clean_text = ' '.join([re.sub(r'<[^>]+>', '', match) for match in text_matches])
            return clean_text
    except Exception as e:
        print(f"Regex метод не работает: {e}")
    
    try:
        # Метод 3: Binary чтение
        with open(file_path, 'rb') as f:
            content = f.read()
        
        content_str = content.decode('utf-8', errors='ignore')
        sentences = re.findall(r'[А-ЯЁа-яё][^.!?]*[.!?]', content_str)
        return ' '.join(sentences[:1000])  # Ограничение для больших файлов
    except Exception as e:
        print(f"❌ Все методы FB2 провалились: {e}")
        return ""

def extract_pdf_text(self, file_path):
    """Извлечение текста из PDF файла"""
    text = ""
    
    # Метод 1: pdfplumber (более точный)
    try:
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages[:20]:  # Ограничение на 20 страниц
                page_text = page.extract_text()
                if page_text:
                    text += page_text + " "
        if text.strip():
            return text
    except Exception as e:
        print(f"pdfplumber ошибка: {e}")
    
    # Метод 2: PyPDF2 (fallback)
    try:
        with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page_num in range(min(20, len(reader.pages))):
                page = reader.pages[page_num]
                text += page.extract_text() + " "
        return text
    except Exception as e:
        print(f"❌ PDF извлечение провалилось: {e}")
        return ""

def extract_doc_text(self, file_path):
    """Извлечение текста из DOC/DOCX файла"""
    try:
        doc = Document(file_path)
        text_parts = []
        
        for paragraph in doc.paragraphs:
            if paragraph.text.strip():
                text_parts.append(paragraph.text.strip())
        
        return ' '.join(text_parts)
    except Exception as e:
        print(f"❌ DOC извлечение провалилось: {e}")
        return ""

# Присваиваем методы классу
SalesBookProcessor.extract_text_from_file = extract_text_from_file
SalesBookProcessor.extract_fb2_text = extract_fb2_text
SalesBookProcessor.extract_pdf_text = extract_pdf_text
SalesBookProcessor.extract_doc_text = extract_doc_text

print("✅ Методы извлечения текста добавлены к классу")


✅ Методы извлечения текста добавлены к классу


In [4]:
# 4. МЕТОДЫ АНАЛИЗА И ОЦЕНКИ ТЕХНИК ПРОДАЖ

def extract_techniques(self, text, book_name):
    """Поиск техник продаж в тексте по regex паттернам"""
    techniques = []
    
    for category, patterns in self.technique_patterns.items():
        for pattern in patterns:
            matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE)
            
            for match in matches:
                if len(match.strip()) > 20:  # Минимальная длина техники
                    technique = {
                        'книга': book_name,
                        'категория': category,
                        'техника': match.strip()[:500],  # Ограничение длины
                        'оценка_качества': self.assess_quality(match.strip(), category)
                    }
                    techniques.append(technique)
    
    return techniques

def assess_quality(self, technique, category):
    """Оценка качества техники от 1 до 10 баллов"""
    score = 5  # Базовая оценка
    
    # Бонусы за специфичность для недвижимости
    if any(word in technique.lower() for word in ['квартира', 'дом', 'недвижимость', 'риелтор']):
        score += 2
    
    # Бонусы за структуру
    if '?' in technique:
        score += 1
    if any(word in technique.lower() for word in ['клиент', 'покупатель', 'продавец']):
        score += 1
    
    # Бонусы по категориям техник
    category_bonuses = {
        'выявление_потребностей': ['нужен', 'требуется', 'хотите', 'ищете'],
        'презентация_объектов': ['выгода', 'преимущество', 'особенность'],
        'работа_с_возражениями': ['понимаю', 'согласен', 'но'],
        'установление_доверия': ['опыт', 'гарантия', 'надежно'],
        'закрытие_сделки': ['решение', 'выбираете', 'готовы']
    }
    
    if category in category_bonuses:
        for bonus_word in category_bonuses[category]:
            if bonus_word in technique.lower():
                score += 0.5
    
    # Штрафы за качество
    if len(technique) < 30:
        score -= 1
    if len(technique) > 300:
        score -= 1
    
    return min(10, max(1, round(score, 1)))

def process_all_books(self):
    """Главный метод обработки всех книг в папке"""
    print(f"\n🔍 Поиск книг в папке: {self.books_dir}")
    
    # Поддерживаемые расширения
    supported_formats = ['.fb2', '.pdf', '.doc', '.docx']
    book_files = []
    
    for format_ext in supported_formats:
        book_files.extend(list(self.books_dir.glob(f"*{format_ext}")))
    
    if not book_files:
        print("❌ Книги не найдены!")
        return
    
    print(f"📚 Найдено книг: {len(book_files)}")
    for book in book_files:
        print(f"  - {book.name}")
    
    all_techniques = []
    processed_books = 0
    
    for book_file in book_files:
        try:
            print(f"\n📖 Обрабатываю: {book_file.name}")
            
            # Извлекаем текст
            text = self.extract_text_from_file(book_file)
            
            if not text or len(text) < 100:
                print(f"❌ Мало текста извлечено из {book_file.name}")
                continue
            
            # Поиск техник
            techniques = self.extract_techniques(text, book_file.stem)
            
            if techniques:
                all_techniques.extend(techniques)
                print(f"✅ Найдено техник: {len(techniques)}")
                processed_books += 1
            else:
                print(f"⚠️  Техники не найдены в {book_file.name}")
                
        except Exception as e:
            print(f"❌ Ошибка при обработке {book_file.name}: {e}")
    
    print(f"\n📊 ИТОГО:")
    print(f"Обработано книг: {processed_books}/{len(book_files)}")
    print(f"Всего техник найдено: {len(all_techniques)}")
    
    if all_techniques:
        # Сохраняем результаты
        self.save_to_csv(all_techniques)
        self.save_summary(all_techniques)
    
    return all_techniques

# Присваиваем методы классу
SalesBookProcessor.extract_techniques = extract_techniques
SalesBookProcessor.assess_quality = assess_quality
SalesBookProcessor.process_all_books = process_all_books

print("✅ Методы анализа и оценки добавлены к классу")


✅ Методы анализа и оценки добавлены к классу


In [5]:
# 5. МЕТОДЫ СОХРАНЕНИЯ ДАННЫХ В CSV И JSON

def save_to_csv(self, techniques):
    """Сохранение техник в CSV файл"""
    output_file = self.data_dir / "quality_techniques.csv"
    
    with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['книга', 'категория', 'техника', 'оценка_качества']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for technique in techniques:
            writer.writerow(technique)
    
    print(f"💾 CSV файл сохранён: {output_file}")
    print(f"📝 Записано техник: {len(techniques)}")
    
    # Показать статистику по категориям
    categories_count = {}
    for technique in techniques:
        category = technique['категория']
        categories_count[category] = categories_count.get(category, 0) + 1
    
    print("\n📊 СТАТИСТИКА ПО КАТЕГОРИЯМ:")
    for category, count in categories_count.items():
        print(f"  {category}: {count} техник")

def save_summary(self, techniques):
    """Сохранение сводки в JSON файл"""
    summary = {
        'дата_обработки': datetime.now().isoformat(),
        'всего_техник': len(techniques),
        'среднее_качество': round(sum(t['оценка_качества'] for t in techniques) / len(techniques), 2),
        'по_категориям': {},
        'по_книгам': {},
        'топ_техники': []
    }
    
    # Статистика по категориям
    for technique in techniques:
        category = technique['категория']
        if category not in summary['по_категориям']:
            summary['по_категориям'][category] = {
                'количество': 0,
                'среднее_качество': 0,
                'оценки': []
            }
        
        summary['по_категориям'][category]['количество'] += 1
        summary['по_категориям'][category]['оценки'].append(technique['оценка_качества'])
    
    # Подсчёт средних оценок по категориям
    for category, data in summary['по_категориям'].items():
        data['среднее_качество'] = round(sum(data['оценки']) / len(data['оценки']), 2)
        del data['оценки']  # Удаляем временный список
    
    # Статистика по книгам
    for technique in techniques:
        book = technique['книга']
        if book not in summary['по_книгам']:
            summary['по_книгам'][book] = {
                'количество': 0,
                'среднее_качество': 0,
                'оценки': []
            }
        
        summary['по_книгам'][book]['количество'] += 1
        summary['по_книгам'][book]['оценки'].append(technique['оценка_качества'])
    
    # Подсчёт средних оценок по книгам
    for book, data in summary['по_книгам'].items():
        data['среднее_качество'] = round(sum(data['оценки']) / len(data['оценки']), 2)
        del data['оценки']  # Удаляем временный список
    
    # Топ-10 лучших техник
    sorted_techniques = sorted(techniques, key=lambda x: x['оценка_качества'], reverse=True)
    summary['топ_техники'] = [
        {
            'техника': t['техника'][:100] + '...' if len(t['техника']) > 100 else t['техника'],
            'категория': t['категория'],
            'книга': t['книга'],
            'оценка': t['оценка_качества']
        }
        for t in sorted_techniques[:10]
    ]
    
    output_file = self.data_dir / "quality_summary.json"
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)
    
    print(f"📋 JSON сводка сохранена: {output_file}")

# Присваиваем методы классу
SalesBookProcessor.save_to_csv = save_to_csv
SalesBookProcessor.save_summary = save_summary

print("✅ Методы сохранения данных добавлены к классу")


✅ Методы сохранения данных добавлены к классу


In [6]:
# 6. СОЗДАНИЕ ЭКЗЕМПЛЯРА И ЗАПУСК ОБРАБОТКИ

# Создаём экземпляр процессора
processor = SalesBookProcessor()

print("\n" + "="*50)
print("🚀 ЗАПУСК ПОЛНОЙ ОБРАБОТКИ ВСЕХ КНИГ")
print("="*50)

# Запускаем обработку всех книг
results = processor.process_all_books()

if results:
    print(f"\n🎯 ОБРАБОТКА ЗАВЕРШЕНА!")
    print(f"✅ Результаты сохранены в папку: {processor.data_dir}")
    print(f"📄 CSV файл: quality_techniques.csv")
    print(f"📋 JSON сводка: quality_summary.json")
    
    # Показываем краткую статистику
    unique_books = len(set(t['книга'] for t in results))
    unique_categories = len(set(t['категория'] for t in results))
    avg_quality = round(sum(t['оценка_качества'] for t in results) / len(results), 2)
    
    print(f"\n📈 КРАТКАЯ СТАТИСТИКА:")
    print(f"  📚 Обработано книг: {unique_books}")
    print(f"  🏷️  Категорий техник: {unique_categories}")
    print(f"  📊 Всего техник найдено: {len(results)}")
    print(f"  ⭐ Среднее качество: {avg_quality}/10")
else:
    print("❌ Обработка завершена без результатов")


✅ Класс SalesBookProcessor инициализирован
📁 Папка с книгами: books
💾 Папка для данных: data
🏷️  Категории техник: 5

🚀 ЗАПУСК ПОЛНОЙ ОБРАБОТКИ ВСЕХ КНИГ

🔍 Поиск книг в папке: books
📚 Найдено книг: 29
  - secret_relrtor.fb2
  - Hopkins_Iskusstvo_torgovat.fb2
  - spin_nil_reckhem.fb2
  - Salihov_Prodazhi-Prostye-recepty-povysheniya-effektivnosti-prodazh-ot-Marata-Salihova.MTXvKQ_RuLit_Me.fb2
  - Treisi_Braian_Effektivnye_metody_prodachi.fb2
  - 48_shagov_k_uspehu.fb2
  - kak-ubezhdat-lyudey.fb2
  - personalizacia_prodag.pdf
  - the_psychology_of_successful_sales.pdf
  - communicating.pdf
  - rodat_bolshe_chem_za_tcelii_mesyatc.pdf
  - 25_most_common_mistakes.pdf
  - sales_how.pdf
  - skazhi_mne_da_preodolenie_vozrazhenij.pdf
  - how_sell_anything.pdf
  - works_objections.pdf
  - text_that_sells.pdf
  - email_persuasion.pdf
  - shkola_prodag.pdf
  - advanced_selling_strategy.pdf
  - kak_stat_millionerom.pdf
  - trening_professionalnih_prodazh.pdf
  - zakryt_sdelku.pdf
  - inie_prodagi.p

In [7]:
# 7. АНАЛИЗ РЕЗУЛЬТАТОВ И ВИЗУАЛИЗАЦИЯ

if results and len(results) > 0:
    # Создаём DataFrame для анализа
    df = pd.DataFrame(results)
    
    print("📊 ДЕТАЛЬНЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ")
    print("="*40)
    
    # Топ-5 книг по количеству техник
    book_counts = df['книга'].value_counts().head()
    print(f"\n🏆 ТОП-5 КНИГ ПО КОЛИЧЕСТВУ ТЕХНИК:")
    for book, count in book_counts.items():
        print(f"  📖 {book}: {count} техник")
    
    # Распределение по категориям
    category_counts = df['категория'].value_counts()
    print(f"\n🏷️  РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ:")
    for category, count in category_counts.items():
        print(f"  • {category}: {count} техник")
    
    # Топ-5 техник по качеству
    top_techniques = df.nlargest(5, 'оценка_качества')
    print(f"\n⭐ ТОП-5 ТЕХНИК ПО КАЧЕСТВУ:")
    for idx, row in top_techniques.iterrows():
        technique_preview = row['техника'][:80] + "..." if len(row['техника']) > 80 else row['техника']
        print(f"  {row['оценка_качества']}/10 | {row['категория']} | {technique_preview}")
    
    # Статистика качества
    print(f"\n📈 СТАТИСТИКА КАЧЕСТВА:")
    print(f"  Среднее качество: {df['оценка_качества'].mean():.2f}/10")
    print(f"  Медиана качества: {df['оценка_качества'].median():.2f}/10")
    print(f"  Лучшая оценка: {df['оценка_качества'].max()}/10")
    print(f"  Худшая оценка: {df['оценка_качества'].min()}/10")
    
    print(f"\n✅ Полные результаты доступны в файлах:")
    print(f"  📄 data/quality_techniques.csv")
    print(f"  📋 data/quality_summary.json")
else:
    print("❌ Нет данных для анализа")


📊 ДЕТАЛЬНЫЙ АНАЛИЗ РЕЗУЛЬТАТОВ

🏆 ТОП-5 КНИГ ПО КОЛИЧЕСТВУ ТЕХНИК:
  📖 Hopkins_Iskusstvo_torgovat: 633 техник
  📖 Treisi_Braian_Effektivnye_metody_prodachi: 344 техник
  📖 spin_nil_reckhem: 331 техник
  📖 kak-ubezhdat-lyudey: 154 техник
  📖 works_objections: 45 техник

🏷️  РАСПРЕДЕЛЕНИЕ ПО КАТЕГОРИЯМ:
  • выявление_потребностей: 1295 техник
  • работа_с_возражениями: 180 техник
  • презентация_объектов: 150 техник
  • установление_доверия: 106 техник
  • закрытие_сделки: 3 техник

⭐ ТОП-5 ТЕХНИК ПО КАЧЕСТВУ:
  9.0/10 | выявление_потребностей | Где публиковать тексты, чтобы привлечь
будущих клиентов84
Плюсы и минусы контент...
  8.5/10 | выявление_потребностей | Где и в каком доме вы хотите жить?
  8.5/10 | презентация_объектов | Перед каждым показом Скотт Спаркс собирал всех потенциальных клиентов и говорил ...
  8.0/10 | выявление_потребностей | Осведомительные вопросы настолько просты и очевидны, что мы часто не ощущаем скр...
  8.0/10 | выявление_потребностей | Что ему может понадоб

In [9]:
pd.set_option('display.max_colwidth', None)  # убирает ограничение ширины колонки
pd.set_option('display.width', None)         # убирает ограничение общей ширины
pd.set_option('display.max_rows', None)      # показать все строки

In [10]:
df = pd.read_csv('/Users/nastyachem/elbrus/realtor/data/quality_techniques.csv')
df.tail(20)

,книга,категория,техника,оценка_качества
1714,trening_professionalnih_prodazh,презентация_объектов,"От того, как они действуют , зависит , что они получают в результате .",5.0
1715,trening_professionalnih_prodazh,презентация_объектов,"И именно профессионализм этих людей часто становится тем \nсамым уникальным конкурентным преимуществом компании , которое ее конкурентам \nтруднее всего перехватить .",5.5
1716,trening_professionalnih_prodazh,презентация_объектов,"Уникальным конкурентным преимуществом коммерческой \nкомпании , которое труднее всего воспроизвести , все чаще становится мастерство ее продавцов .",5.5
1717,trening_professionalnih_prodazh,презентация_объектов,"Книга , которая станет вашим уникальным конкурентным преимуществом .",5.5
1718,trening_professionalnih_prodazh,работа_с_возражениями,Малейшее сомнение пилота может привести к \nкатастрофе .,5.0
1719,trening_professionalnih_prodazh,работа_с_возражениями,"КОНТРОЛЬ ЗА ЭМОЦИОНАЛЬНЫМ СОСТОЯНИЕМ КЛИЕНТА \nПараллельно с движением по функциональным этапам двигайтесь по этапам эмоциональным : Страх , Сомнение , \nБезразличие , Интерес , Желание , Согласие .",6.5
1720,trening_professionalnih_prodazh,работа_с_возражениями,"Пройти \nчерез страх , недоверие , сомнение .",5.0
1721,trening_professionalnih_prodazh,установление_доверия,"Вот \nпочему одна из главных задач менеджеров при организации работы продавцов — это \nпередать им веру в продаваемые продукты и услуги , а также в надежность и стабильность \nкомпании в целом .",5.5
1722,trening_professionalnih_prodazh,установление_доверия,"самых важных фактов о вашей компании , которые вызовут к ней доверие .",5.0
1723,trening_professionalnih_prodazh,установление_доверия,"Первый ряд оборонительных сооружений , которые клиент выстраивает \nвокруг себя, — это страх и недоверие по отношению к вам или к компании , \nкоторую вы представляете .",6.0
